In [ ]:
# def visualize(self, rawtext: str, default_to_notebook: bool = True, mode: str = 'rawtext'):
#     r"""Visualize text from input

#     Problem: Not working with GPT2Tokenizer

#     Arguments:

#             rawtext (:obj: `str`):
#                 Raw text input.

#             default_to_notebook(:obj: `bool`, `optional`, defaults to :obj: `True`):
#                 output to notebook (or html)

#             mode (:obj: `str`, `optional`, defaults to :obj: `rawtext`):
#                 Options: `text`, `rawtext`

#     """
#     if mode == 'text':
#         text = self.encode_whitespaces(rawtext)
#         self.visualizer(text, default_to_notebook=default_to_notebook)
#     elif mode == 'rawtext':
#         self.visualizer(rawtext, default_to_notebook=default_to_notebook)

In [1]:
%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds


In [2]:
cd ..

/home/huawei123/kwx1991442/code-classification


load data

In [3]:
from src.codeparrot.dataloader import UITestsLoader, ClassificationCollator
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch import Generator

In [4]:
dataset = UITestsLoader(mode='train')
splitgenerator = Generator().manual_seed(42)
train_set, val_set = random_split(dataset, lengths=[0.7, 0.3], generator=splitgenerator)

In [5]:
collate_fn = ClassificationCollator(dataset.classes)
loader_train = DataLoader(train_set, batch_size=2, collate_fn=collate_fn)
loader_val = DataLoader(val_set, batch_size=2, collate_fn=collate_fn)

load model

In [6]:
from src.params import PATH_CODEPARROT
from transformers import (
    GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification, 
    AutoModelForCausalLM, AutoModelForSequenceClassification,
    AdamW, get_linear_schedule_with_warmup)

model_name_or_path = PATH_CODEPARROT
n_labels = len(dataset.classes)
n_labels

2023-08-16 19:48:06.586712: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 19:48:07.383761: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


307

In [7]:
# # load auto model
# model_config = AutoModelForCausalLM.from_pretrained(
#     model_name_or_path, num_labels=n_labels)
# model = AutoModelForSequenceClassification.from_pretrained(
#     pretrained_model_name_or_path=model_name_or_path, config=model_config, num_labels=n_labels)
# model.resize_token_embeddings(len(collate_fn.tokenizer))
# model.config.pad_token_id = model.config.eos_token_id


# # load gpt2 model
model_config = GPT2Config.from_pretrained(
    pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)
model = GPT2ForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=model_name_or_path, config=model_config)
model.resize_token_embeddings(len(collate_fn.tokenizer))
model.config.pad_token_id = model.config.eos_token_id

Some weights of the model checkpoint at data/codeparrot-small were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at data/codeparrot-small and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


setup

In [8]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [9]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )

/home/huawei123/kwx1991442/venv/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


train

In [10]:
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

In [11]:
class Trainer(object):
    model : GPT2ForSequenceClassification
    optimizer : AdamW
    loader_train : DataLoader
    loader_val : DataLoader
    loader_inf : DataLoader
    n_epochs : int
    def __init__(self, 
        model, optimizer,
        loader_train, loader_val, loader_inf = None,
        n_epochs = 4,
        experiment_name = 'test'
        ) -> None:
        self.model = model
        self.loader_train, self.loader_val = loader_train, loader_val
        self.loader_inf = loader_inf

        self.n_epochs = n_epochs
        self.optimizer = optimizer
        self.scheduler = self.setup_scheduler()

        self.writer : SummaryWriter = self.setup_tensorboard(experiment_name)
        self.log_loss = {'train' : [], 'validation' : []}
        self.log_accu = {'train' : [], 'validation' : []}

    def setup_scheduler(self):
        scheduler = get_linear_schedule_with_warmup(self.optimizer, 
            num_warmup_steps = 0, 
            num_training_steps = self.n_epochs * len(self.loader_train))
        return scheduler

    def setup_tensorboard(self, experiment_name):
        now = datetime.now().strftime(f"%d-%b-%H-%M-%S")
        folder = 'saved_data/tb-log'
        if not os.path.exists(folder): os.makedirs(folder)
        writer = SummaryWriter(os.path.join(folder), comment=f"{experiment_name}_{now}")
        layout = {
            "metrics": {
                "loss": ["Multiline", ["loss/train", "loss/validation"]],
                "accuracy": ["Multiline", ["accuracy/train", "accuracy/validation"]],
                "epoch_loss": ["Multiline", ["epoch_loss/train", "epoch_loss/validation"]],
                "epoch_accuracy": ["Multiline", ["epoch_accuracy/train", "epoch_accuracy/validation"]],
            },
        }
        writer.add_custom_scalars(layout)
        return writer

    def train_one_epoch(self, n_epoch):
        predicted_labels = []
        true_labels = []
        total_loss = 0

        self.model.train()
        for n_batch, (sequences, labels) in tqdm(enumerate(self.loader_train), total=len(self.loader_train)):

            self.model.zero_grad()

            outputs = self.model(**sequences, labels=labels)
            loss, logits = outputs[:2]

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            
            self.optimizer.step()
            self.scheduler.step()

            # log
            logits = logits.detach().cpu().numpy()
            predicted_labels += logits.argmax(axis=-1).flatten().tolist()
            true_labels += labels.numpy().flatten().tolist()
            total_loss += loss.item()

            accu = accuracy_score(labels.numpy().flatten().tolist(), logits.argmax(axis=-1).flatten().tolist())
            self.writer.add_scalar("loss/train", loss.item(), n_epoch * len(self.loader_train) + n_batch)
            self.writer.add_scalar("accuracy/train", accu, n_epoch * len(self.loader_train) + n_batch)
            print(f"train \tloss : {loss.item()}\taccu : {accu}\t{n_epoch * len(self.loader_train) + n_batch}")

        avg_epoch_loss = total_loss / len(self.loader_train)
        self.log(true_labels, predicted_labels, avg_epoch_loss, n_epoch, 'train')

    def evaluate(self, n_epoch):
        predicted_labels = []
        true_labels = []
        total_loss = 0

        self.model.eval()
        for n_batch, (sequences, labels) in tqdm(enumerate(self.loader_val), total=len(self.loader_val)):

            with torch.no_grad():
                outputs = self.model(**sequences, labels=labels)
                loss, logits = outputs[:2]

            # log
            logits = logits.detach().cpu().numpy()
            predicted_labels += logits.argmax(axis=-1).flatten().tolist()
            true_labels += labels.numpy().flatten().tolist()
            total_loss += loss.item()

            accu = accuracy_score(labels.numpy().flatten().tolist(), logits.argmax(axis=-1).flatten().tolist())
            self.writer.add_scalar("loss/validation", loss.item(), n_epoch * len(self.loader_train) + n_batch)
            self.writer.add_scalar("accuracy/validation", accu, n_epoch * len(self.loader_train) + n_batch)

        avg_epoch_loss = total_loss / len(self.loader_train)
        self.log(true_labels, predicted_labels, avg_epoch_loss, n_epoch, 'validation')

    def log(self, true_labels, predicted_labels, arg_epoch_loss, n_epoch, mode='train'):
        accu = accuracy_score(true_labels, predicted_labels)
        self.log_accu[mode].append(accu)
        self.log_loss[mode].append(arg_epoch_loss)
        print(f"{mode}\tloss : {arg_epoch_loss}\taccu : {accu}")

        self.writer.add_scalar(f"epoch_loss/{mode}", arg_epoch_loss, n_epoch)
        self.writer.add_scalar(f"epoch_accuracy/{mode}", accu, n_epoch)

    def train(self):
        for n_epoch in tqdm(range(self.n_epochs)):
            self.train_one_epoch(n_epoch)
            self.evaluate(n_epoch)

In [12]:
trainer = Trainer(model, optimizer, loader_train, loader_val)

In [13]:
trainer.train()

  0%|          | 0/4 [00:00<?, ?it/s]

train 	loss : 6.545134544372559	accu : 0.0	0


train 	loss : 5.692564964294434	accu : 0.0	1


train 	loss : 4.668007850646973	accu : 0.0	2


train 	loss : 5.310105323791504	accu : 0.0	3


...

train 	loss : 0.006840990390628576	accu : 1.0	9042


train 	loss : 1.2476807832717896	accu : 0.5	9043


train 	loss : 0.013306084088981152	accu : 1.0	9044


 50%|█████     | 2/4 [16:50:26<16:50:26, 30313.24s/it]


KeyboardInterrupt: 

In [14]:
trainer.log_accu

{'train': [0.2744356160928238, 0.616471181737924],
 'validation': [0.4855797527957622, 0.6071218363743378]}

In [15]:
trainer

In [16]:
trainer.evaluate(1)

  1%|          | 12/1699 [00:28<1:06:36,  2.37s/it]


KeyboardInterrupt: 